In [1]:
from ultralytics import YOLO
import cv2

In [3]:
 # Load YOLO model
model = YOLO('yolov5su.pt')  # Pretrained YOLOv5 model

# Open video file or webcam feed
cap = cv2.VideoCapture("C:\\Users\\91997\\Pictures\\Camera Roll\\Main.mp4")
#cap = cv2.VideoCapture(0)
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define VideoWriter to save output
out = cv2.VideoWriter('output_yolo.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Object detection
    results = model(frame)

    # Render detected objects on the frame
    annotated_frame = results[0].plot()

    # Save the frame to the output video
    out.write(annotated_frame)

    # Display the frame
    cv2.imshow('YOLOv5 Object Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 143.1ms
Speed: 5.8ms preprocess, 143.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 197.7ms
Speed: 8.1ms preprocess, 197.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 146.9ms
Speed: 4.9ms preprocess, 146.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 150.3ms
Speed: 2.9ms preprocess, 150.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 157.4ms
Speed: 4.0ms preprocess, 157.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 128.7ms
Speed: 3.1ms preprocess, 128.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 car, 105.3ms
Speed: 1.0ms preprocess, 105.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 79.1ms
Speed: 2.0ms preprocess, 79.1ms inference, 0.0ms postprocess 

In [8]:
import torch
from torchvision import transforms
from torchvision.models.detection import ssd300_vgg16
import cv2

In [18]:
# Load SSD model
model = ssd300_vgg16(pretrained=True).eval()

# Define labels for COCO dataset
LABELS = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane", "bus",
    "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter",
    "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra",
    "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis",
    "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard",
    "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
    "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table",
    "toilet", "TV", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave",
    "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
    "teddy bear", "hair drier", "toothbrush"
]

# Open video source
cap = cv2.VideoCapture("C:\\Users\\91997\\Pictures\\Camera Roll\\Main.mp4")  # Replace with video file path for pre-recorded video

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize video writer
out = cv2.VideoWriter(
    'output_ssd.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height)
)

# Transform for model input
transform = T.Compose([T.ToTensor()])

# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    input_tensor = transform(frame).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        detections = model(input_tensor)[0]

    # Annotate detected objects on the frame
    for i in range(len(detections['scores'])):
        score = detections['scores'][i].item()
        if score > 0.5:  # Confidence threshold
            box = detections['boxes'][i].numpy().astype(int)
            label = LABELS[int(detections['labels'][i])]

            # Draw bounding box
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            # Add label and score
            cv2.putText(frame, f"{label} {score:.2f}", (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save annotated frame to output video
    out.write(frame)

    # Display the frame
    cv2.imshow("SSD Object Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
